In [1]:
from mydataset import TRANSFORMER_DATA_MINDS,TRANSFORMER_ALL_DATA_MINDS
from mymodel import mymodel, Discriminator, backboneDiscriminator
from myloss import advLoss
from tqdm import tqdm
import random
import numpy
import os 
import mindspore.dataset as ds
import mindspore.nn as nn
import mindspore.ops as ops
from mindspore import Model,ParameterTuple
import math
import numpy as np
from mindspore import dtype as mstype
import mindspore.nn as nn
import mindspore.ops as ops
from mindspore import Model
from mindspore import dataset as ds
from mindspore.nn import LossBase
from mindspore.common.initializer import Normal
from mindspore.train.callback import LossMonitor,SummaryCollector
from mindspore import set_context,context
from mindspore.train.callback import Callback
from mindspore import Tensor
from mindspore.dataset import TupleIterator,DictIterator
import pickle

def prepareData(source_list,target_list,target_test_names):
    s_path = '/Domain-Adaptive-Remaining-Useful-Life-Prediction-with-Transformer/RUL/s_data.dat'
    t_path = '/Domain-Adaptive-Remaining-Useful-Life-Prediction-with-Transformer/RUL/t_data.dat'
    tt_path = '/Domain-Adaptive-Remaining-Useful-Life-Prediction-with-Transformer/RUL/t_data_test.dat'
    try:
        with open(s_path, 'rb') as sf:
            s_data = pickle.load(sf)
        with open(t_path, 'rb') as tf:
            t_data = pickle.load(tf)
        with open(tt_path, 'rb') as ttf:
            t_data_test = pickle.load(tf)
    except Exception as e:
        print(e)
        s_data = TRANSFORMER_ALL_DATA_MINDS(source_list, seq_len)
        t_data = TRANSFORMER_ALL_DATA_MINDS(target_list, seq_len)
        t_data_test = TRANSFORMER_ALL_DATA_MINDS(target_test_names, seq_len)
        with open(s_path, "wb") as sf:
            pickle.dump(s_data, sf, protocol=pickle.HIGHEST_PROTOCOL)
        with open(t_path, "wb") as tf:
            pickle.dump(t_data, tf, protocol=pickle.HIGHEST_PROTOCOL)
        with open(tt_path, "wb") as ttf:
            pickle.dump(t_data_test, ttf, protocol=pickle.HIGHEST_PROTOCOL)
        print('saved all data...')
    return s_data,t_data,t_data_test

    
class MywithLossCell(nn.Cell):
    def __init__(self,net, D1, D2,loss_fn, auto_prefix=False):
        super(MywithLossCell, self).__init__()
        self.net = net
        self.D1 = D1
        self.D2 = D2
        self.loss_fn = loss_fn

    def construct(self,s_input, s_lb,s_msk, t_input=None, t_msk=None):
        s_features, s_out = self.net(s_input, s_msk)
        t_features, t_out = self.net(t_input, t_msk)
        s_out = s_out.squeeze(2)
        t_out = t_out.squeeze(2)
        s_domain_bkb = self.D2(s_features[0])
        t_domain_bkb = self.D2(t_features[0])
        s_domain_out = self.D1(s_out)
        t_domain_out = self.D1(t_out)
        return self.loss_fn(s_out,s_lb
                            ,s_domain_bkb.squeeze(1), t_domain_bkb.squeeze(1)
                            ,s_domain_out.squeeze(1), t_domain_out.squeeze(1))
    


class EvalCallBack(Callback):
    # 进行评估的类，实例化后传入训练的callback中进行计算
    # ref: https://www.cnblogs.com/skytier/p/16638657.html
    # result_eval = {"epoch": [], "acc": [], "loss": []}
    # eval_cb = EvalCallBack(net, ds_val, EVAL_PER_EPOCH, result_eval)
    def __init__(self, net, eval_per_epoch, epoch_per_eval):
        self.net = net
        self.eval_per_epoch = eval_per_epoch
        self.epoch_per_eval = epoch_per_eval
        # epoch_per_eval：记录验证模型的精度和相应的epoch数，其数据形式为{"epoch": [], "acc": []}

    def epoch_end(self, run_context):
        cb_param = run_context.original_args()
        cur_epoch = cb_param.cur_epoch_num
        if cur_epoch % self.eval_per_epoch == 0:
            # begin eval:
            tot = 0
            for i in tqdm(target_test_names):
                pred_sum, pred_cnt = Tensor(np.zeros(800)), Tensor(np.zeros(800))
                # 记录结果 （800维？）
                valid_data = TRANSFORMER_DATA_MINDS(i, seq_len)
                # 变换一项结果
                data_len = len(valid_data)
                sampler = ds.RandomSampler()
                valid_dataset = ds.GeneratorDataset(valid_data,sampler=sampler,column_names=['input','label','msk'])
                valid_dataset = valid_dataset.batch(batch_size=len(valid_data),drop_remainder=True)
                vd_iter = valid_dataset.create_tuple_iterator()
                d = next(vd_iter)
                input, lbl, msk = d[0], d[1], d[2]
                outputs = self.net(input,msk)
                # return of WithEvalCell:
                #   return loss, (net)outputs, label
                out = outputs[1]
                out = out.squeeze(2)
                for j in range(data_len):
                    if j < seq_len-1:
                        pred_sum[:j+1] += out[j, -(j+1):]
                        pred_cnt[:j+1] += 1
                    elif j <= data_len-seq_len:
                        pred_sum[j-seq_len+1:j+1] += out[j]
                        pred_cnt[j-seq_len+1:j+1] += 1
                    else:
                        pred_sum[data_len-seq_len+1-(data_len-j):data_len-seq_len+1] += out[j, :(data_len-j)]
                        pred_cnt[data_len-seq_len+1-(data_len-j):data_len-seq_len+1] 
                
                truth = Tensor([lbl[j,-1].asnumpy() for j in range(len(lbl)-seq_len+1)], dtype=mstype.float32)
                pred_sum, pred_cnt = pred_sum[:data_len-seq_len+1], pred_cnt[:data_len-seq_len+1]
                pred = pred_sum/pred_cnt
                mse = (ops.pow(pred-truth, 2)).sum()
                rmse = math.sqrt(mse/data_len)
                tot += rmse
            Rc = 130
            mse_res = tot*Rc/len(valid_list)
            self.epoch_per_eval["mse"].append(mse_res)
            print('eval score: ' + str(mse_res))
            
            

# 以下cell是model的组成成分，用于在callback 中调用，定义行为
# https://www.hiascend.com/app-forum/topic-detail/0224103731274466014
class CustomWithEvalCell(nn.Cell):
    """自定义多标签评估网络"""

    def __init__(self, network):
        super(CustomWithEvalCell, self).__init__(auto_prefix=False)
        self.network = network
        # network is mymodel, receive src as fellow:
        '''
        def construct(self, src, attn_msk=None):
            src = self.pos_encoder(src)
            # output1 = self.transformer_encoder(src, attn_msk, key_msk)
            # problem？
            attn_msk = self.trans_mask_generator(attn_msk)
            output1 = self.transformer_encoder(src, attn_msk)
            # return the multiple result..
            output2 = self.decoder(output1[0])
            return output1, output2
        '''
    def construct(self, target_test_names):
        output = self.network(data)
        return output, label1, label2
    


class MERGED_DATA():
    def __init__(self,s_data,t_data) -> None:
        self.s_data = s_data
        self.t_data = t_data
    
    def __len__(self):
        return min(len(self.s_data),len(self.t_data))
    
    def __getitem__(self,index):
        # return self.s_data[index]+(self.t_data[index][0],self.t_data[index][2])
        return (self.s_data[index][0],self.s_data[index][1],self.s_data[index][2],self.t_data[index][0],self.t_data[index][2])
        # ['s_input', 's_lb', 's_msk','t_input', 't_msk']


class MultipleLoss(LossBase):
    def __init__(self, reduction='mean'):
        super(MultipleLoss, self).__init__(reduction)
        self.mseLoss = nn.MSELoss()
        self.feaLoss = advLoss()
        self.outLoss = advLoss()
        self.a = 0.1
        self.b = 0.5
        self.allLoss = 0.0

    def construct(self, s_r, s_lb, s_bkb, t_bkb, s_out, t_out):
        loss1 = self.mseLoss(s_r, s_lb)
        loss2 = self.feaLoss(s_bkb, t_bkb)
        loss3 = self.outLoss(s_out, t_out)
        return loss1 + self.a*loss2 + self.b*loss3


In [2]:
# prepare data
seq_len = 70
target= 'FD002'
source = 'FD003'
epoches = 240
os.chdir('/Domain-Adaptive-Remaining-Useful-Life-Prediction-with-Transformer/')
batch_size = 100
a = 0.1
b = 0.5
source_list = numpy.loadtxt("save/"+source+"/train"+source+".txt", dtype=str).tolist()
target_list = numpy.loadtxt("save/"+target+"/train"+target+".txt", dtype=str).tolist()
valid_list = numpy.loadtxt("save/"+target+"/test"+target+".txt", dtype=str).tolist()
# len(v) = 47
a_list = numpy.loadtxt("save/"+target+"/valid"+target+".txt", dtype=str).tolist()
# len(a_list) 31
target_test_names = valid_list + a_list
# 78 files
# target_test_names = target_test_names[]
minl = min(len(source_list), len(target_list))
s_data,t_data,t_data_test = prepareData(source_list,target_list,target_test_names)

peek of closed file


  4%|▍         | 3/78 [00:01<00:34,  2.16it/s]


saved all data...


In [3]:
len(s_data)
l = t_data[0]
for feature in l:
    print(Tensor.mean(feature))
    print(Tensor.var(feature))
    print(feature)
    print(feature.shape)
    


0.00971056
0.0072242646
[[0.      0.      0.      ... 0.      0.      0.     ]
 [0.      0.      0.      ... 0.      0.      0.     ]
 [0.      0.      0.      ... 0.      0.      0.     ]
 ...
 [0.      0.      0.      ... 0.      0.      0.     ]
 [0.      0.      0.      ... 0.      0.      0.     ]
 [0.23803 0.29715 1.      ... 1.      0.62894 0.62891]]
(70, 24)
1.0
0.0
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
(70,)
0.9857143
0.014081633
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0.]
(70,)


In [4]:
source_list[0]

'FD003-56.txt'

In [5]:
# test model output..
net = mymodel(max_len=seq_len)

In [6]:
sf,sm,sl = s_data[0]
sf = sf.unsqueeze(0)
sl = sl.unsqueeze(0)
sm = sm.unsqueeze(0)
net.set_train()
o1,o2 = net(sf,sm)

In [7]:
type(o1)

tuple

In [8]:
len(o1)

2

In [9]:
print(o1[0])

[[[-0.9211623   1.0781664  -0.9084619  ...  1.104544   -0.92165077
    1.1025901 ]
  [-0.77566534 -0.7869045  -0.7668695  ... -0.76980144 -0.75856227
    1.3030918 ]
  [-0.90833426  1.0835211  -0.91761464 ...  1.1040356  -0.9234759
    1.0962206 ]
  ...
  [-0.9119707   1.0804917  -0.91148233 ...  1.1049091  -0.92662114
    1.0961188 ]
  [-0.8492003   1.1810099  -0.84382296 ...  1.1927423  -0.8628882
   -0.8423564 ]
  [-0.68635905 -1.6641768  -0.02287679 ...  1.6381541  -0.66635376
    0.8652681 ]]]


In [10]:
Tensor.mean(o1[0])

Tensor(shape=[], dtype=Float32, value= -2.27065e-09)

In [11]:
Tensor.var(o1[0])

Tensor(shape=[], dtype=Float32, value= 0.99999)

In [12]:
for o in o1:
    print(type(o))
    print(len(o))

<class 'mindspore.common.tensor.Tensor'>
1
<class 'tuple'>
2


In [13]:
len(o1[1][0])

2

In [14]:
o1[1][0][1]

Tensor(shape=[1, 8, 70, 3], dtype=Float16, value=
[[[[ 6.3660e-02,  3.3150e-03, -3.5339e-02],
   [ 2.1957e-02,  1.4854e-02, -2.5272e-03],
   [ 6.2500e-02,  1.4305e-02, -3.0731e-02],
   ...
   [ 5.0079e-02,  1.5686e-02, -4.5685e-02],
   [ 4.4586e-02, -6.0234e-03, -3.1311e-02],
   [ 6.8115e-02,  4.1901e-02, -6.8604e-02]],
  [[-3.1982e-02, -2.1805e-02,  2.1088e-02],
   [-2.0309e-02, -3.8574e-02,  8.6517e-03],
   [ 1.5991e-02, -2.4261e-02,  1.2865e-03],
   ...
   [-6.8855e-03,  1.4519e-02,  1.2459e-02],
   [ 1.5106e-02, -1.9943e-02, -8.2855e-03],
   [ 5.4382e-02, -6.0944e-02,  7.9102e-02]],
  [[ 5.9605e-05,  2.3071e-02, -8.2825e-02],
   [ 1.0246e-02,  1.2726e-02, -3.2867e-02],
   [ 4.7211e-02,  2.1545e-02, -6.4697e-02],
   ...
   [ 3.4546e-02,  3.3844e-02, -7.0251e-02],
   [ 4.6326e-02,  2.1423e-02, -4.0222e-02],
   [ 2.4353e-02,  3.4119e-02, -1.4844e-01]],
  ...
  [[-3.1052e-02,  2.2034e-02,  6.2378e-02],
   [-9.7809e-03,  2.3422e-02,  4.2969e-02],
   [-4.3297e-03, -3.1834e-03,  4.3671e-0

In [15]:
o1[1][0][1]

Tensor(shape=[1, 8, 70, 3], dtype=Float16, value=
[[[[ 6.3660e-02,  3.3150e-03, -3.5339e-02],
   [ 2.1957e-02,  1.4854e-02, -2.5272e-03],
   [ 6.2500e-02,  1.4305e-02, -3.0731e-02],
   ...
   [ 5.0079e-02,  1.5686e-02, -4.5685e-02],
   [ 4.4586e-02, -6.0234e-03, -3.1311e-02],
   [ 6.8115e-02,  4.1901e-02, -6.8604e-02]],
  [[-3.1982e-02, -2.1805e-02,  2.1088e-02],
   [-2.0309e-02, -3.8574e-02,  8.6517e-03],
   [ 1.5991e-02, -2.4261e-02,  1.2865e-03],
   ...
   [-6.8855e-03,  1.4519e-02,  1.2459e-02],
   [ 1.5106e-02, -1.9943e-02, -8.2855e-03],
   [ 5.4382e-02, -6.0944e-02,  7.9102e-02]],
  [[ 5.9605e-05,  2.3071e-02, -8.2825e-02],
   [ 1.0246e-02,  1.2726e-02, -3.2867e-02],
   [ 4.7211e-02,  2.1545e-02, -6.4697e-02],
   ...
   [ 3.4546e-02,  3.3844e-02, -7.0251e-02],
   [ 4.6326e-02,  2.1423e-02, -4.0222e-02],
   [ 2.4353e-02,  3.4119e-02, -1.4844e-01]],
  ...
  [[-3.1052e-02,  2.2034e-02,  6.2378e-02],
   [-9.7809e-03,  2.3422e-02,  4.2969e-02],
   [-4.3297e-03, -3.1834e-03,  4.3671e-0

In [16]:
a = Tensor(1.0)

In [17]:
a

Tensor(shape=[], dtype=Float32, value= 1)

In [18]:
print(f"{a} {a} {a}")

1.0 1.0 1.0


In [20]:
net.trainable_params()

[Parameter (name=transformer_encoder.blocks.0.layernorm1.gamma, shape=(24,), dtype=Float32, requires_grad=True),
 Parameter (name=transformer_encoder.blocks.0.layernorm1.beta, shape=(24,), dtype=Float32, requires_grad=True),
 Parameter (name=transformer_encoder.blocks.0.layernorm2.gamma, shape=(24,), dtype=Float32, requires_grad=True),
 Parameter (name=transformer_encoder.blocks.0.layernorm2.beta, shape=(24,), dtype=Float32, requires_grad=True),
 Parameter (name=transformer_encoder.blocks.0.attention.projection.weight, shape=(24, 24), dtype=Float32, requires_grad=True),
 Parameter (name=transformer_encoder.blocks.0.attention.projection.bias, shape=(24,), dtype=Float32, requires_grad=True),
 Parameter (name=transformer_encoder.blocks.0.attention.dense1.weight, shape=(24, 24), dtype=Float32, requires_grad=True),
 Parameter (name=transformer_encoder.blocks.0.attention.dense1.bias, shape=(24,), dtype=Float32, requires_grad=True),
 Parameter (name=transformer_encoder.blocks.0.attention.dens

In [24]:
D1 = Discriminator(in_features=seq_len)
D2 = backboneDiscriminator(seq_len)
D1.trainable_params()

[Parameter (name=li.0.weight, shape=(512, 70), dtype=Float32, requires_grad=True),
 Parameter (name=li.0.bias, shape=(512,), dtype=Float32, requires_grad=True),
 Parameter (name=li.1.gamma, shape=(512,), dtype=Float32, requires_grad=True),
 Parameter (name=li.1.beta, shape=(512,), dtype=Float32, requires_grad=True),
 Parameter (name=li.3.weight, shape=(1, 512), dtype=Float32, requires_grad=True),
 Parameter (name=li.3.bias, shape=(1,), dtype=Float32, requires_grad=True)]

In [25]:
D2.trainable_params()

[Parameter (name=li1.weight, shape=(1, 24), dtype=Float32, requires_grad=True),
 Parameter (name=li1.bias, shape=(1,), dtype=Float32, requires_grad=True),
 Parameter (name=li2.0.weight, shape=(512, 70), dtype=Float32, requires_grad=True),
 Parameter (name=li2.0.bias, shape=(512,), dtype=Float32, requires_grad=True),
 Parameter (name=li2.1.gamma, shape=(512,), dtype=Float32, requires_grad=True),
 Parameter (name=li2.1.beta, shape=(512,), dtype=Float32, requires_grad=True),
 Parameter (name=li2.3.weight, shape=(1, 512), dtype=Float32, requires_grad=True),
 Parameter (name=li2.3.bias, shape=(1,), dtype=Float32, requires_grad=True)]

In [26]:
loss_func = MultipleLoss()

In [27]:
loss_net = MywithLossCell(net,D1,D2,loss_func)

In [28]:
loss_net.trainable_params()

[Parameter (name=net.transformer_encoder.blocks.0.layernorm1.gamma, shape=(24,), dtype=Float32, requires_grad=True),
 Parameter (name=net.transformer_encoder.blocks.0.layernorm1.beta, shape=(24,), dtype=Float32, requires_grad=True),
 Parameter (name=net.transformer_encoder.blocks.0.layernorm2.gamma, shape=(24,), dtype=Float32, requires_grad=True),
 Parameter (name=net.transformer_encoder.blocks.0.layernorm2.beta, shape=(24,), dtype=Float32, requires_grad=True),
 Parameter (name=net.transformer_encoder.blocks.0.attention.projection.weight, shape=(24, 24), dtype=Float32, requires_grad=True),
 Parameter (name=net.transformer_encoder.blocks.0.attention.projection.bias, shape=(24,), dtype=Float32, requires_grad=True),
 Parameter (name=net.transformer_encoder.blocks.0.attention.dense1.weight, shape=(24, 24), dtype=Float32, requires_grad=True),
 Parameter (name=net.transformer_encoder.blocks.0.attention.dense1.bias, shape=(24,), dtype=Float32, requires_grad=True),
 Parameter (name=net.transfo

In [29]:

sampler = ds.RandomSampler()
dataset = ds.GeneratorDataset(MERGED_DATA(s_data,t_data),sampler=sampler,column_names=['s_input', 's_lb', 's_msk','t_input', 't_msk'])
dataset = dataset.batch(batch_size=batch_size,drop_remainder=True)
rtl = TupleIterator(dataset)
loss_func = MultipleLoss()
net = mymodel(max_len=seq_len,batch_size=batch_size)
D1 = Discriminator(in_features=seq_len)
D2 = backboneDiscriminator(seq_len)
loss_net = MywithLossCell(net,D1,D2,loss_func)
learning_rate = nn.piecewise_constant_lr([80,160,240],[0.5,0.25,0.125])
opt = nn.SGD(net.trainable_params()+D1.trainable_params()+D2.trainable_params()
            ,learning_rate=learning_rate)
result_eval = {"mse": []}
eval_cb = EvalCallBack(net, 100, result_eval)
summary_collector = SummaryCollector(summary_dir = './RUL/summary_dir',collect_freq=1)
print("before model")

# model = Model(network=loss_net, optimizer=opt)
# FORMAT two dataset into one.


loss_net.set_train(True)
# model.train(epoch=100, train_dataset=dataset, callbacks=[LossMonitor(per_print_times=10),eval_cb,summary_collector])
# model.train(epoch=1000, train_dataset=dataset, callbacks=[LossMonitor(per_print_times=10)])


before model


MywithLossCell<
  (net): mymodel<
    (pos_encoder): PositionalEncoding<
      (dropout): Dropout<keep_prob=0.9>
      >
    (transformer_encoder): TransformerEncoder<
      (blocks): CellList<
        (0): TransformerEncoderLayer<
          (layernorm1): _LayerNorm<>
          (layernorm2): _LayerNorm<>
          (attention): MultiHeadAttention<
            (projection): _Linear<>
            (dropout): Dropout<keep_prob=0.9>
            (prob_dropout): Dropout<keep_prob=0.9>
            (softmax): Softmax<>
            (softmax_3d): Softmax<>
            (dense1): _Linear<>
            (dense2): _Linear<>
            (dense3): _Linear<>
            >
          (output): FeedForward<
            (mapping): _Linear<
              (activation): GELU<>
              >
            (projection): _Linear<>
            (dropout): Dropout<keep_prob=0.9>
            (dropout_3d): Dropout<keep_prob=0.9>
            (dropout_4d): Dropout<keep_prob=0.9>
            >
          >
        (1): Tran

In [39]:

# todo:每个epoch的时候进行一次数据的打乱。
from mindspore.dataset import TupleIterator,DictIterator
rtl = TupleIterator(dataset)
class TrainOneStepCell(nn.Cell):
    def __init__(self, network, optimizer, sens=1.0):
        """参数初始化"""
        super(TrainOneStepCell, self).__init__(auto_prefix=False)
        self.network = network
        # 使用tuple包装weight
        self.weights = ParameterTuple(network.trainable_params())
        self.optimizer = optimizer
        # 定义梯度函数
        self.grad = ops.GradOperation(get_by_list=True)

    def construct(self, data):
        """构建训练过程"""
        weights = self.weights
        loss = self.network(data[0],data[1],data[2],data[3],data[4])
        # 为反向传播设定系数
        grads = self.grad(self.network, weights)(data[0],data[1],data[2],data[3],data[4])
        return loss, self.optimizer(grads)
    
train_model = TrainOneStepCell(loss_net,opt)
# 重构训练循环：https://gitee.com/mindspore/docs/blob/r2.0.0-alpha/tutorials/application/source_zh_cn/cv/resnet50.ipynb
# 参考2：https://www.mindspore.cn/tutorial/zh-CN/r1.2/intermediate/custom/train.html
for j in range(10):
    for i,d in tqdm(enumerate(rtl)):
        train_model(d)

0it [00:00, ?it/s][ERROR] DEVICE(165792,7f5ff27f4700,python):2023-05-24-11:51:37.331.072 [mindspore/ccsrc/runtime/pynative/async/async_queue.cc:75] WorkerLoop] Run task failed, error msg:The pointer[runtime_info] is null.

----------------------------------------------------
- C++ Call Stack: (For framework developers)
----------------------------------------------------
mindspore/ccsrc/runtime/pynative/run_op_helper.cc:534 LaunchKernels

[ERROR] DEVICE(165792,7f5ff27f4700,python):2023-05-24-11:51:37.331.212 [mindspore/ccsrc/runtime/pynative/async/async_queue.cc:75] WorkerLoop] Run task failed, error msg:The pointer[runtime_info] is null.

----------------------------------------------------
- C++ Call Stack: (For framework developers)
----------------------------------------------------
mindspore/ccsrc/runtime/pynative/run_op_helper.cc:534 LaunchKernels

0it [00:00, ?it/s]


RuntimeError: The pointer[runtime_info] is null.

----------------------------------------------------
- C++ Call Stack: (For framework developers)
----------------------------------------------------
mindspore/ccsrc/runtime/pynative/run_op_helper.cc:534 LaunchKernels


[Tensor(shape=[100, 70, 24], dtype=Float32, value=
[[[ 6.04650021e-01,  1.53850004e-01,  1.00000000e+00 ...  1.00000000e+00,  8.57140005e-01,  5.12939990e-01],
  [ 5.11629999e-01,  6.92309976e-01,  1.00000000e+00 ...  1.00000000e+00,  6.78569973e-01,  7.36800015e-01],
  [ 6.91860020e-01,  1.53850004e-01,  1.00000000e+00 ...  1.00000000e+00,  6.36900008e-01,  8.08520019e-01],
  ...
  [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00 ...  0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
  [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00 ...  0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
  [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00 ...  0.00000000e+00,  0.00000000e+00,  0.00000000e+00]],
 [[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00 ...  0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
  [ 5.87210000e-01,  3.07689995e-01,  1.00000000e+00 ...  1.00000000e+00,  5.95239997e-01,  5.24720013e-01],
  [ 2.90699989e-01,  9.23080027e-01,  1.00000000e+00 ...  1.00000000e+

In [41]:
[1.0,0.5]

TypeError: can't multiply sequence by non-int of type 'float'